In [48]:
import numpy as np
import pandas as pd
import xarray as xr

from os.path import isfile
from glob import glob
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

In [49]:
obdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/'
profdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/profiles/'

In [50]:
flist = glob(obdir + 'clean/*.pd')

# This can be a manual site list if desired
site_list = [f.split('/')[-1].split('_')[0] for f in flist]
site_list

['CLN', 'ALTA', 'AGD', 'BCC', 'SLB', 'PVC']

# Build into a loop over all stations

In [51]:
# for site in site_list:
site = 'CLN'

### Obtain the metadata

In [52]:
meta_file = glob(obdir + '*Metadata*.xlsx')[0]
metadata = pd.read_excel(meta_file).set_index('code').loc[site]

# Determine the lat, lon of the site from the metadata
site_lat, site_lon = metadata['lat'], metadata['lon']
site_elev = metadata['elevation_m']

metadata

name               Alta Collins
owner            Alta Ski Lifts
city                       Alta
state                        UT
lat                     40.5763
lon                    -111.638
elevation_m                2945
start                       NaN
end                         NaN
interval                     12
n_events                    NaN
auto_manual              Manual
snow_type      Snow Board (HSN)
swe_type                    NaN
notes                          
Name: CLN, dtype: object

In [65]:
# Import the era5_orog file to check the lat/lon grid
era_dir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/'
era5_orog_file = era_dir + 'era5_orog.nc'
era5_orog = xr.open_dataset(era5_orog_file)['z'].isel(time=0)
era5_orog = era5_orog.rename({'latitude':'lat', 'longitude':'lon'})
era5_lat, era5_lon = era5_orog['lat'], era5_orog['lon']

# Find the index of the correct lat lon
idx1d = (np.abs(era5_lon - site_lon) + np.abs(era5_lat - site_lat))
idx = np.unravel_index(np.argmin(idx1d, axis=None), idx1d.shape)

# Subset and convert gpm to m
era5_g = 9.80665
era5_orog = era5_orog.isel(lat=idx[1], lon=idx[0])/era5_g

era5_lat, era5_lon = era5_orog['lat'].values, era5_orog['lon'].values
era5_elev = era5_orog.values

print('ERA5 Profile:\n%.2f %.2f %.2f\n'%(era5_lat, era5_lon, era5_elev))

era5_prof_file = 'era5prof_{}N_{}W.nc'.format(
    era5_orog['lat'].values, abs(era5_orog['lon'].values))

if isfile(era_dir + 'profiles/' + era5_prof_file):
    era5_prof = xr.open_dataset(era_dir + 'profiles/' + era5_prof_file).load()
    print(era5_prof)
    
else:
    print('ERA5 Profile Missing:\n%s'%era5_prof_file)

ERA5 Profile:
40.50 -111.75 1967.42

<xarray.Dataset>
Dimensions:  (level: 23, time: 346224)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2019-06-30T23:00:00
  * level    (level) float64 200.0 225.0 250.0 300.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    Q        (time, level) float32 8.430327e-06 2.2606831e-05 ... 0.0056080474
    T        (time, level) float32 213.56236 213.23013 ... 302.55365 303.83582
    U        (time, level) float32 32.258484 38.37195 ... -0.18589783
    V        (time, level) float32 2.0954742 2.2194977 ... 4.7066345 4.6651
    VO       (time, level) float32 6.204704e-06 ... -0.000121165765
    W        (time, level) float32 0.017477036 0.077495575 ... -0.11104393
    Z        (time, level) float32 115597.81 108389.75 ... 3440.6726 1319.3672
    R        (time, level) float32 23.817059 75.24865 ... 28.302814 28.428883
    U100M    (time) float32 -0.80430603 -0.3778534 ... 0.10606384 -0.1478424
    V100M    (time) float32 3.1638489 2.974456

In [74]:
# df = era5_prof.to_dataframe()
# df[:10]

In [75]:
# df.reset_index().set_index(['time', 'level']).sort_index()